# Importing Libraries

In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#  Importing Data Set


In [3]:
data=pd.read_csv(r"C:\Users\yaksh\OneDrive\Desktop\BIA\DATA\movies\movies.csv")
data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [85]:
data.shape

(9742, 3)

The main aim of this project is to build a movie recomendation engine.
-> In the data set we can see that movieid, title and genres are columns of the  dataset
-> MOVIEID gives the identification number of a particular movie
-> TITLE gives the name of the movie and the year in which the movie belongs to.
-> GENRES gives the category of the move in which it belongs to

# Null Values

In [4]:
data.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

We can see that there are no null values given in the data set

# Data Preprocessing

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [7]:
data.iloc[0]

movieId                                              1
title                                 Toy Story (1995)
genres     Adventure|Animation|Children|Comedy|Fantasy
Name: 0, dtype: object

In [72]:
data['genres']=data['genres'].astype(str)

In [73]:
def converter(obj):
    c=obj.split('|')
    return c

In [74]:
data1=data.copy()

In [75]:
data1['genres']=data1['genres'].apply(converter)

In [76]:
data1.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [77]:
data1['genres']=data1['genres'].apply(lambda x: [i.replace("-","") for i in x])

In [78]:
def converter2(obj):
    x=" ".join(obj)
    return x

In [79]:
data2=data1.copy()
data2['genres']=data2['genres'].apply(converter2)

In [80]:
data2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [81]:
def movie(obj):
    c=obj.split(" ")
    s=''
    for i in range(len(c)-1):
        s=s+c[i]+" "
    return s.strip()

In [82]:
data3=data2.copy()
data3['title']=data3['title'].apply(movie)
data3.head()

,movieId,title,genres
0,1,Toy Story,Adventure Animation Children Comedy Fantasy
1,2,Jumanji,Adventure Children Fantasy
2,3,Grumpier Old Men,Comedy Romance
3,4,Waiting to Exhale,Comedy Drama Romance
4,5,Father of the Bride Part II,Comedy


In [83]:
data3['genres'].iloc[0]

'Adventure Animation Children Comedy Fantasy'

# Recomendation System

In general a recomendation system is designed using three principle they are content based filtring, collebrative based filtring and hybrid filtring
CONTENT BASED FILTRING: Here depending upon the regular trends of the content a user is being watched recomendations are provided.
COLLEBRAIVE BASED FILTRING: In this depending upon the similarities of the users the recomendations are provided.
HYBRID FILTRING: It involves both content based and collebrative based filtring techniques big tech gaints like google and meta uses this technique.

In this project we are using content based filtring where we use tags to determine the content here in our data set genre are the tags which we will be considering and with help of vectorization we will find the similar movies and recommend them to the users. For vectorizarion we use the technique called as BAG OF WORDS

BAG OF WORDS: we will concatenate all the tags and find the highest frequency words in which we eliminate the stop words. Then we will count no.of words present in each tag for these highest frequency words. By which we can form vectors for each movie and we can give the closest movie vectors as the recommendations.

# Vectorization

In [89]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=100)

In [91]:
vectors=cv.fit_transform(data3['genres']).toarray()

In [92]:
vectors

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [95]:
# Cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

In [96]:
similarity=cosine_similarity(vectors)

In [97]:
similarity

array([[1.        , 0.77459667, 0.31622777, ..., 0.        , 0.31622777,
        0.4472136 ],
       [0.77459667, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31622777, 0.        , 1.        , ..., 0.        , 0.        ,
        0.70710678],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.31622777, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.4472136 , 0.        , 0.70710678, ..., 0.        , 0.        ,
        1.        ]])

In [110]:
def recomend(movie):
    index=data3[data3['title']==movie].index[0]
    distance=sorted(list(enumerate(similarity[index])),reverse=True,key=lambda x:x[1])
    for i in distance[1:6]:
        print(data3.iloc[i[0]].title)

In [111]:
recomend('Toy Story')

Antz
Toy Story 2
Adventures of Rocky and Bullwinkle, The
Emperor's New Groove, The
Monsters, Inc.


In [112]:
recomend('Dreamscape')

Night of the Living Dead
Event Horizon
Mimic
Them!
Village of the Damned


In the above function we we will find the index of the given movie and locate it in the similarity array then we will sort the array and print the top 2 to 6 movies from data frame depending upon on the index.

CONCLUSION: The 'recomend' function takes a movie name as an input and it will be giving five recomendations of movies to watch